In [2]:
import pandas as pd
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np

# Загрузка данных
users = pd.read_csv('data/users.csv')
items = pd.read_csv('data/items.csv')
reviews = pd.read_csv('data/reviews.csv')


In [3]:
G = nx.Graph()

In [4]:
for _, row in items.iterrows():
    G.add_node(row['detail_id'], type='place', name=row['name'], rating=row['rating'])

for _, row in reviews.iterrows():
    G.add_node(row['profile_url'], type='user')
    G.add_edge(row['profile_url'], row['detail_id'], weight=row['mark'])

# Добавление рёбер между местами на основе схожести
# Формируем матрицу признаков для мест
place_features = items[['latitude', 'longitude', 'rating'] +
                       [col for col in items.columns if col.startswith('tags_') or col.startswith('cuisines_')]]
place_ids = items['detail_id'].values
scaler = StandardScaler()
place_features_scaled = scaler.fit_transform(place_features.fillna(0))

similarity_matrix = cosine_similarity(place_features_scaled)
threshold = 0.5

for i, place_id1 in enumerate(place_ids):
    for j, place_id2 in enumerate(place_ids):
        if i != j and similarity_matrix[i, j] > threshold:
            G.add_edge(place_id1, place_id2, weight=similarity_matrix[i, j])


In [5]:
def recommend_route(user_id, graph, top_n=5):
    if user_id not in graph:
        return "Пользователь не найден в графе."

    user_places = [neighbor for neighbor in graph.neighbors(user_id) if graph.nodes[neighbor]['type'] == 'place']

    similar_places = []
    for place_id in user_places:
        for neighbor in graph.neighbors(place_id):
            if graph.nodes[neighbor]['type'] == 'place' and neighbor not in user_places:
                similar_places.append((neighbor, graph[place_id][neighbor]['weight']))

    similar_places = sorted(similar_places, key=lambda x: -x[1])[:top_n]
    recommended_places = [place for place, _ in similar_places]

    return recommended_places

In [6]:
user_id = '/Profile/yalmaree'
recommended_places = recommend_route(user_id, G)
print("Рекомендуемые места для пользователя:", recommended_places)

Рекомендуемые места для пользователя: [12245093, 6977474, 6493701, 27484126, 8477924]
